# Day 51

Updating my rankings for teams based on number of fantasy points allowed to the position through Week 14.

In [6]:
import pandas as pd
import sqlite3

# Create database connection
conn = sqlite3.connect('../../data/db/database.db')

In [7]:
season = 2022
week = 14

In [8]:
query = f"""
WITH fantasy_totals_by_team AS (
    -- Get the total fantasy points scored per team, per position, per week through Week 13
    SELECT
        recent_team,
        position,
        week,
        ROUND(SUM(fantasy_points), 2) as tot_pts,
        ROUND(SUM(fantasy_points_ppr), 2) as tot_pts_ppr
    FROM weekly
    WHERE season = {season}
        AND week <= {week}
        AND position in ('QB', 'RB', 'WR', 'TE')
    GROUP BY recent_team, position, week),
-- Get schedule info through week 13
    home_games AS (
        SELECT 
            game_id,
            season,
            week, 
            home_team AS team,
            away_team AS opp_team
        FROM schedules
        WHERE season = {season}
            AND week <= {week}
    ),
    away_games AS (
       SELECT 
            game_id,
            season,
            week, 
            away_team AS team,
            home_team AS opp_team
        FROM schedules
        WHERE season = {season}
            AND week <= {week}
    ),
    stacked AS (
        SELECT *
        FROM home_games
        UNION
        SELECT *
        FROM away_games
    ),
-- Join the two temp datasets
-- Every team will have it's total fantasy points per team appended to it
joined_data AS (
    SELECT
        game_id,
        season,
        stacked.week,
        team,
        opp_team,
        position,
        tot_pts,
        tot_pts_ppr
    FROM stacked
    LEFT JOIN fantasy_totals_by_team
        ON fantasy_totals_by_team.recent_team = stacked.team
        AND fantasy_totals_by_team.week = stacked.week),
-- Only use opp_team to get the points scored scored against
total_pts_scored_against AS (
    SELECT
        opp_team,
        position,
        SUM(tot_pts) AS tot_pts_scored_against,
        SUM(tot_pts_ppr) AS tot_pts_ppr_scored_against
    FROM joined_data
    GROUP BY opp_team, position),
-- Utilize RANK() to get the season-level rankings
season_rankings AS (
    SELECT
        opp_team AS team,
        position,
        RANK() OVER(PARTITION BY position ORDER BY tot_pts_scored_against DESC) AS r_pts_scored_against,
        RANK() OVER(PARTITION BY position ORDER BY tot_pts_ppr_scored_against DESC) AS r_pts_ppr_scored_against
    FROM total_pts_scored_against
    ORDER BY position, r_pts_ppr_scored_against)
SELECT *
FROM season_rankings
"""

df = pd.read_sql(query, conn)
df

,team,position,r_pts_scored_against,r_pts_ppr_scored_against
0,DET,QB,1,1
1,TEN,QB,2,2
2,KC,QB,3,3
3,MIA,QB,4,4
4,LV,QB,5,5
...,...,...,...,...
123,HOU,WR,28,28
124,CIN,WR,29,29
125,NYJ,WR,30,30
126,IND,WR,31,31


## Standard Scoring

In [11]:
standard = df.drop('r_pts_ppr_scored_against', axis=1).sort_values('r_pts_scored_against')

for pos in ['QB', 'RB', 'WR', 'TE']:
    print(standard.query(f"(position == '{pos}') & (r_pts_scored_against <= 5 | r_pts_scored_against >= 28)"))

   team position  r_pts_scored_against
0   DET       QB                     1
1   TEN       QB                     2
2    KC       QB                     3
3   MIA       QB                     4
4    LV       QB                     5
27   SF       QB                    28
28  DEN       QB                    29
29  DAL       QB                    30
30  HOU       QB                    31
31  PHI       QB                    32
   team position  r_pts_scored_against
32  HOU       RB                     1
33  SEA       RB                     2
34  CHI       RB                     3
37  LAC       RB                     4
35  CLE       RB                     5
60  DAL       RB                    28
56  BAL       RB                    29
62   NE       RB                    30
61  TEN       RB                    31
63   SF       RB                    32
    team position  r_pts_scored_against
97   TEN       WR                     1
96   MIN       WR                     2
98   DET       WR     

## PPR Scoring

In [10]:
ppr = df.drop('r_pts_scored_against', axis=1).sort_values('r_pts_ppr_scored_against')

for pos in ['QB', 'RB', 'WR', 'TE']:
    print(ppr.query(f"(position == '{pos}') & (r_pts_ppr_scored_against <= 3 | r_pts_ppr_scored_against >= 30)"))

   team position  r_pts_ppr_scored_against
0   DET       QB                         1
1   TEN       QB                         2
2    KC       QB                         3
29  DAL       QB                        30
30  HOU       QB                        31
31  PHI       QB                        32
   team position  r_pts_ppr_scored_against
32  HOU       RB                         1
33  SEA       RB                         2
34  CHI       RB                         3
61  TEN       RB                        30
62   NE       RB                        31
63   SF       RB                        32
    team position  r_pts_ppr_scored_against
96   MIN       WR                         1
97   TEN       WR                         2
98   DET       WR                         3
125  NYJ       WR                        30
126  IND       WR                        31
127  DEN       WR                        32
   team position  r_pts_ppr_scored_against
64  ARI       TE                         1
65  